# Normalising the Data
In this notebook we perform the differential scaling. This means depending on the feature type we scale the values differently. Some values are not scaled (dimension reduced versions of features), most are standard scaled regularly, but the intensity features are scaled on a per plate basis.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import organoid_prediction_python as opp

home_directory = r"C:\Users\savill\OneDrive\Documents\PhD Jesse\Embryonic_organoid_prediction\Processing dataset for Paper\TLS_2D_morphostate_investigation/"
table_location = home_directory + "tabular_data"

data_complete = pd.read_csv(f"{table_location}/All_Samples_Feature_Space_All_Features_48h,72h,96h.csv").set_index(["Run","Plate","ID"])
data_complete

Unnamed: 0  048h_raw_BF_AreaShape_area  \
Run Plate ID                                            
RR  1     A1            0                51429.057934   
          A2            1                46107.767989   
          A3            2                54482.752732   
          A4            3                46999.688747   
          A5            4                50680.751535   
...                   ...                         ...   
TR  2     H8          762                47634.615388   
          H9          763                45616.455707   
          H10         764                42336.001394   
          H11         765                40484.132024   
          H12         766                44648.948444   

               048h_raw_BF_AreaShape_aspect_ratio  \
Run Plate ID                                        
RR  1     A1                             1.036110   
          A2                             1.018480   
          A3                             1.070369   
          A4                             1.054378   
          A5                             1.067224   
...                                           ...   
TR  2     H8                             1.074280   
          H9                             1.141554   
          H10                            1.017190   
          H11                            1.034860   
          H12                            1.028101   

               048h_raw_BF_AreaShape_axis_major_length  \
Run Plate ID                                             
RR  1     A1                                260.694837   
          A2                                244.753205   
          A3                                272.945953   
          A4                                251.337713   
          A5                                262.580886   
...                                                ...   
TR  2     H8                                255.443622   
          H9                                257.800200   
          H10                               234.372899   
          H11                               231.114030   
          H12                               241.897406   

               048h_raw_BF_AreaShape_axis_minor_length  \
Run Plate ID                                             
RR  1     A1                                251.609209   
          A2                                240.312256   
          A3                                255.001802   
          A4                                238.375438   
          A5                                246.040977   
...                                                ...   
TR  2     H8                                237.781139   
          H9                                225.832628   
          H10                               230.412135   
          H11                               223.328768   
          H12                               235.285546   

               048h_raw_BF_AreaShape_eccentricity  \
Run Plate ID                                        
RR  1     A1                             0.261703   
          A2                             0.189631   
          A3                             0.356599   
          A4                             0.316996   
          A5                             0.349302   
...                                           ...   
TR  2     H8                             0.365387   
          H9                             0.482314   
          H10                            0.183066   
          H11                            0.257365   
          H12                            0.232206   

               048h_raw_BF_AreaShape_feret_diameter_max  \
Run Plate ID                                              
RR  1     A1                                 266.724612   
          A2                                 252.890770   
          A3                                 280.334245   
          A4                                 256.319703   
          A5                 

## Defining which features are sclaed by which method

In [2]:
no_scaling = ["UMAP_","PC_","Location"]
per_plate_scaling = ["RadialDistribution","Intensity"]

keys_no_scaling = [key for key in data_complete.keys() if any([word in key for word in no_scaling])]
keys_per_plate_scaling = [key for key in data_complete.keys() if any([word in key for word in per_plate_scaling]) and not any([word in key for word in no_scaling])]
keys_normal_scaling = [key for key in data_complete.keys() if "AreaShape" in key and not any([word in key for word in no_scaling + ["loco"]])]
keys_group_scaling = [key for key in data_complete.keys() if "loco" in key and not any([word in key for word in no_scaling])]

#keys_per_plate_scaling

['072h_raw_CH__bf_mask_Intensity_IntegratedIntensity',
 '072h_raw_CH__bf_mask_Intensity_IntegratedIntensityEdge',
 '072h_raw_CH__bf_mask_Intensity_LowerQuartileIntensity',
 '072h_raw_CH__bf_mask_Intensity_MADIntensity',
 '072h_raw_CH__bf_mask_Intensity_MassDisplacement',
 '072h_raw_CH__bf_mask_Intensity_MaxIntensity',
 '072h_raw_CH__bf_mask_Intensity_MaxIntensityEdge',
 '072h_raw_CH__bf_mask_Intensity_MeanIntensity',
 '072h_raw_CH__bf_mask_Intensity_MeanIntensityEdge',
 '072h_raw_CH__bf_mask_Intensity_MedianIntensity',
 '072h_raw_CH__bf_mask_Intensity_MinIntensity',
 '072h_raw_CH__bf_mask_Intensity_MinIntensityEdge',
 '072h_raw_CH__bf_mask_Intensity_StdIntensity',
 '072h_raw_CH__bf_mask_Intensity_StdIntensityEdge',
 '072h_raw_CH__bf_mask_Intensity_UpperQuartileIntensity',
 '072h_raw_CH__bf_mask_RadialDistribution_FracAtD_1of9',
 '072h_raw_CH__bf_mask_RadialDistribution_FracAtD_2of9',
 '072h_raw_CH__bf_mask_RadialDistribution_FracAtD_3of9',
 '072h_raw_CH__bf_mask_RadialDistribution_Frac

## Performing the differential standard scaling

In [3]:
diff_scaling_df = opp.differential_standard_scaling(data_complete,keys_normal_scaling,keys_per_plate_scaling,keys_no_scaling,keys_group_scaling)
diff_scaling_df = diff_scaling_df.reindex(sorted(diff_scaling_df.columns), axis=1)

## Saving the result

In [4]:
diff_scaling_df.to_csv(f"{table_location}/All_Samples_Feature_Space_All_Features_48h,72h,96h_Normalised_per_Timepoint.csv")

# Stacking time data
Here we are stacking all the features from different timepoints and adding an Hour index, so that we can scale across all timepoints

In [5]:
def stack_time_data(dataframe, hours = ["48","72","96"]):
    time_keys = [[key for key in dataframe.keys() if key[1:].startswith(str(hour))]+["Run","Plate","ID"] for hour in hours]
    dataframes_separated = [dataframe[key_list].rename(columns={key:key[5:] for key in key_list if key not in ["Run","Plate","ID"]}) for key_list in time_keys]
    
    out = []
    for frame,hour in zip(dataframes_separated,hours):
        frame["hour"] = np.full(len(frame),hour)
        out.append(frame)
        
    return pd.concat(out,axis=0,ignore_index=True)

data_complete_no_index = pd.read_csv(f"{table_location}/All_Samples_Feature_Space_All_Features_48h,72h,96h.csv")
out = stack_time_data(data_complete_no_index)
out

,raw_BF_AreaShape_area,raw_BF_AreaShape_aspect_ratio,raw_BF_AreaShape_axis_major_length,raw_BF_AreaShape_axis_minor_length,raw_BF_AreaShape_eccentricity,raw_BF_AreaShape_feret_diameter_max,raw_BF_AreaShape_perimeter,raw_BF_AreaShape_solidity,str_BF_AreaShape_area,str_BF_AreaShape_aspect_ratio,...,raw_CH_bra_mask_RadialDistribution_RadialCV_3of9,raw_CH_bra_mask_RadialDistribution_RadialCV_4of9,raw_CH_bra_mask_RadialDistribution_RadialCV_5of9,raw_CH_bra_mask_RadialDistribution_RadialCV_6of9,raw_CH_bra_mask_RadialDistribution_RadialCV_7of9,raw_CH_bra_mask_RadialDistribution_RadialCV_8of9,raw_CH_bra_mask_RadialDistribution_RadialCV_9of9,str_CH_AreaShape_Bra_MajorAxis_Polarisation,str_CH_AreaShape_Bra_MinorAxis_Polarisation,raw_CH_AreaShape_Bra_AreaFraction
0,51429.057934,1.036110,260.694837,251.609209,0.261703,266.724612,848.528995,0.985088,51519.761740,1.053506,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,46107.767989,1.018480,244.753205,240.312256,0.189631,252.890770,807.094108,0.984030,46032.181484,1.017855,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,54482.752732,1.070369,272.945953,255.001802,0.356599,280.334245,885.017919,0.983893,54898.478509,1.089076,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,46999.688747,1.054378,251.337713,238.375438,0.316996,256.319703,817.033443,0.985420,46886.308989,1.057124,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,50680.751535,1.067224,262.580886,246.040977,0.349302,267.361474,844.917216,0.985450,50348.170914,1.057777,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2296,111134.838154,1.142154,405.046069,354.633456,0.483148,426.097154,1281.489393,0.983741,115224.068070,1.265925,...,0.059606,0.046989,0.045001,0.054103,0.057371,0.056742,0.073117,0.010821,0.026928,0.940896
2297,125088.106959,1.240381,449.765600,362.602919,0.591637,478.386100,1389.507526,0.973528,126962.652281,1.322848,...,0.036139,0.049559,0.084494,0.141751,0.151588,0.113339,0.089259,0.035616,0.026081,0.873950
2298,128799.404350,1.177665,444.001992,377.019027,0.528171,461.129001,1400.733636,0.970995,127408.612659,1.199837,...,0.028084,0.021513,0.035351,0.054787,0.073556,0.079270,0.073310,0.009338,0.004076,0.940728
2299,119313.297985,1.451500,473.874839,326.472498,0.724816,463.075495,1343.308177,0.971624,122858.305064,1.614141,...,0.067871,0.067214,0.058404,0.056358,0.053544,0.047907,0.031259,0.012343,0.042325,0.958505


## Performing the differential standard scaling for Stacked time data

In [6]:
diff_scaled_timepoints_combined = opp.differential_standard_scaling(
    out.set_index(["hour","Run","Plate","ID"]),
    list(set([key[5:] for key in keys_normal_scaling])),
    list(set([key[5:] for key in keys_per_plate_scaling])),
    list(set([key[5:] for key in keys_no_scaling if "UMAP_" not in key and "PC_" not in key])),
    list(set([key[5:] for key in keys_group_scaling])),
    ["Run","Plate"]
)
diff_scaled_timepoints_combined

raw_CH__bf_mask_Location_Center_X  \
hour Run Plate ID                                       
48   RR  1     A1                                 NaN   
               A2                                 NaN   
               A3                                 NaN   
               A4                                 NaN   
               A5                                 NaN   
...                                               ...   
96   TR  2     H8                          369.320547   
               H9                          516.973956   
               H10                         402.688087   
               H11                         421.072347   
               H12                         528.471119   

                    raw_CH_bra_mask_Location_CenterMassIntensity_X  \
hour Run Plate ID                                                    
48   RR  1     A1                                              NaN   
               A2                                              NaN   
               A3                                              NaN   
               A4                                              NaN   
               A5                                              NaN   
...                                                            ...   
96   TR  2     H8                                       368.556524   
               H9                                       522.330168   
               H10                                      401.366129   
               H11                                      421.649370   
               H12                                      527.968536   

                    raw_CH_bra_mask_Location_MaxIntensity_X  \
hour Run Plate ID                                             
48   RR  1     A1                                       NaN   
               A2                                       NaN   
               A3                                       NaN   
               A4                                       NaN   
               A5                                       NaN   
...                                                     ...   
96   TR  2     H8                                     376.0   
               H9                                     523.0   
               H10                                    396.0   
               H11                                    427.0   
               H12                                    534.0   

                    raw_CH_bra_mask_Location_MaxIntensity_Y  \
hour Run Plate ID                                             
48   RR  1     A1                                       NaN   
               A2                                       NaN   
               A3                                       NaN   
               A4                                       NaN   
               A5                                       NaN   
...                                                     ...   
96   TR  2     H8                                     318.0   
               H9                                     230.0   
               H10                                    289.0   
               H11                                    293.0   
               H12                                    234.0   

                    raw_CH__bf_mask_Location_CenterMassIntensity_Y  \
hour Run Plate ID                                                    
48   RR  1     A1                                              NaN   
               A2                                              NaN   
               A3                                              NaN   
               A4                                              NaN   
               A5                                              NaN   
...                                                            ...   
96   TR  2     H8                                       315.886732   
               H9                                       228.593520   
               H10         

In [8]:
diff_scaled_timepoints_combined = diff_scaled_timepoints_combined.reindex(sorted(diff_scaled_timepoints_combined.columns), axis=1)
diff_scaled_timepoints_combined.to_csv(f"{table_location}/All_Samples_Feature_Space_All_Features_48h,72h,96h_Normalised_across_Timepoints.csv")
diff_scaled_timepoints_combined

raw_BF_AreaShape_area  raw_BF_AreaShape_aspect_ratio  \
hour Run Plate ID                                                          
48   RR  1     A1               -0.786269                      -0.602748   
               A2               -1.021382                      -0.728751   
               A3               -0.651346                      -0.357900   
               A4               -0.981974                      -0.472190   
               A5               -0.819332                      -0.380374   
...                                   ...                            ...   
96   TR  2     H8                1.851740                       0.155152   
               H9                2.468244                       0.857179   
               H10               2.632222                       0.408947   
               H11               2.213093                       2.366057   
               H12               2.416813                       1.017937   

                    raw_BF_AreaShape_axis_major_length  \
hour Run Plate ID                                        
48   RR  1     A1                            -0.804054   
               A2                            -1.052827   
               A3                            -0.612872   
               A4                            -0.950074   
               A5                            -0.774622   
...                                                ...   
96   TR  2     H8                             1.448585   
               H9                             2.146445   
               H10                            2.056502   
               H11                            2.522676   
               H12                            2.117180   

                    raw_BF_AreaShape_axis_minor_length  \
hour Run Plate ID                                        
48   RR  1     A1                            -0.680943   
               A2                            -0.975465   
               A3                            -0.592495   
               A4                            -1.025959   
               A5                            -0.826112   
...                                                ...   
96   TR  2     H8                             2.004995   
               H9                             2.212767   
               H10                            2.588608   
               H11                            1.270813   
               H12                            2.005679   

                    raw_BF_AreaShape_eccentricity  \
hour Run Plate ID                                   
48   RR  1     A1                       -0.839800   
               A2                       -1.307770   
               A3                       -0.223631   
               A4                       -0.480778   
               A5                       -0.271012   
...                                           ...   
96   TR  2     H8                        0.598064   
               H9                        1.302495   
               H10                       0.890400   
               H11                       2.167236   
               H12                       1.426414   

                    raw_BF_AreaShape_feret_diameter_max  \
hour Run Plate ID                                         
48   RR  1     A1                             -0.833971   
               A2                             -1.046449   
               A3                             -0.624938   
               A4                             -0.993783   
               A5                             -0.824190   
...                                                 ...   
96   TR  2     H8                              1.613871   
               H9                              2.416990   
               H10                             2.151934   
               H11                             2.181831   
               H12                             2.145512   

                    raw_BF_AreaShape_perimeter  raw_BF_AreaShap